In [20]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

def gen_blobs():
    classes = 6
    m = 800
    std = 0.4
    centers = np.array([[-1, 0], [1, 0], [0, 1], [0, -1], [-2,1],[-2,-1]])
    X, y = make_blobs(n_samples=m, centers=centers, cluster_std=std)
    return (X, y, centers, classes, std)

In [21]:
X, y, centers, classes, std = gen_blobs()
X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.5, random_state=1)
X_cv, X_test, y_cv, y_test = train_test_split(X_, y_, test_size=0.5, random_state=1)
print(X_train.shape, X_cv.shape, X_test.shape)

(400, 2) (200, 2) (200, 2)


In [22]:
def eval_cat_err(y, yhat):
    m = y.shape[0]
    err = 0
    for i in range(m):
        if y[i] != yhat[i]:
            err += 1
    return err/m

#### We should start with a complex model

In [23]:
tf.random.set_seed(1234)
model = Sequential(
    [
        Dense(120, activation="relu"),
        Dense(40, activation="relu"),
        Dense(6, activation="linear")
    ], name = "complex"
)
model.compile(
    optimizer=Adam(learning_rate=0.01), 
    loss=SparseCategoricalCrossentropy(from_logits=True)
)
model.fit(X_train, y_train, epochs=1000)
model.summary()

Epoch 1/1000
 1/13 [=>............................] - ETA: 4s - loss: 1.7700

2024-07-15 20:34:18.388454: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 0s 10ms/step - loss: 1.0983
Epoch 2/1000
13/13 [==============================] - 0s 5ms/step - loss: 0.3854
Epoch 3/1000
13/13 [==============================] - 0s 6ms/step - loss: 0.3304
Epoch 4/1000
13/13 [==============================] - 0s 7ms/step - loss: 0.3050
Epoch 5/1000
13/13 [==============================] - 0s 11ms/step - loss: 0.2959
Epoch 6/1000
13/13 [==============================] - 0s 6ms/step - loss: 0.2767
Epoch 7/1000
13/13 [==============================] - 0s 6ms/step - loss: 0.2307
Epoch 8/1000
13/13 [==============================] - 0s 6ms/step - loss: 0.1936
Epoch 9/1000
13/13 [==============================] - 0s 6ms/step - loss: 0.1942
Epoch 10/1000
13/13 [==============================] - 0s 6ms/step - loss: 0.1868
Epoch 11/1000
13/13 [==============================] - 0s 6ms/step - loss: 0.1999
Epoch 12/1000
13/13 [==============================] - 0s 7ms/step - loss: 0.1973
Epoch 13/1000
13/13 [===============

In [24]:
model_predict = lambda Xl: np.argmax(tf.nn.softmax(model.predict(Xl)), axis = 1)
training_cerr_complex = eval_cat_err(y_train, model_predict(X_train))
cv_cerr_complex = eval_cat_err(y_cv, model_predict(X_cv))

print(f"categorization error, training, complex model: {training_cerr_complex:0.3f}")
print(f"categorization error, cv,       complex model: {cv_cerr_complex:0.3f}")

7/7 [==============================] - 0s 4ms/step
categorization error, training, complex model: 0.010
categorization error, cv,       complex model: 0.090


2024-07-15 20:35:37.638326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


#### Now let's try a simple model

In [25]:
tf.random.set_seed(1234)
model_s = Sequential(
    [
        Dense(6, activation="relu"),
        Dense(6, activation="linear")
    ], name = "simple"
)
model_s.compile(
    optimizer = Adam(learning_rate=0.01),
    loss = SparseCategoricalCrossentropy(from_logits=True)
)
model_s.fit(X_train, y_train, epochs=1000)
model_s.summary()

Epoch 1/1000
10/13 [======================>.......] - ETA: 0s - loss: 1.9589

2024-07-15 20:36:00.478659: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 0s 8ms/step - loss: 1.9301
Epoch 2/1000
13/13 [==============================] - 0s 5ms/step - loss: 1.6413
Epoch 3/1000
13/13 [==============================] - 0s 5ms/step - loss: 1.4809
Epoch 4/1000
13/13 [==============================] - 0s 5ms/step - loss: 1.3301
Epoch 5/1000
13/13 [==============================] - 0s 5ms/step - loss: 1.1715
Epoch 6/1000
13/13 [==============================] - 0s 5ms/step - loss: 1.0154
Epoch 7/1000
13/13 [==============================] - 0s 5ms/step - loss: 0.8828
Epoch 8/1000
13/13 [==============================] - 0s 5ms/step - loss: 0.7858
Epoch 9/1000
13/13 [==============================] - 0s 4ms/step - loss: 0.7083
Epoch 10/1000
13/13 [==============================] - 0s 4ms/step - loss: 0.6478
Epoch 11/1000
13/13 [==============================] - 0s 5ms/step - loss: 0.5963
Epoch 12/1000
13/13 [==============================] - 0s 5ms/step - loss: 0.5550
Epoch 13/1000
13/13 [=================

In [26]:
model_predict_s = lambda Xl: np.argmax(tf.nn.softmax(model_s.predict(Xl)), axis = 1)
training_cerr_simple = eval_cat_err(y_train, model_predict_s(X_train))
cv_cerr_simple = eval_cat_err(y_cv, model_predict_s(X_cv))

print(f"categorization error, training, simple model, {training_cerr_simple:0.3f}, complex model: {training_cerr_complex:0.3f}" )
print(f"categorization error, cv,       simple model, {cv_cerr_simple:0.3f}, complex model: {cv_cerr_complex:0.3f}" )

7/7 [==============================] - 0s 4ms/step
categorization error, training, simple model, 0.052, complex model: 0.010
categorization error, cv,       simple model, 0.065, complex model: 0.090


2024-07-15 20:37:14.012361: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Our simple model has a little higher classification error on training data but does better on cross-validation data than the more complex model.

#### We can apply regularization to the more complex model

In [28]:
tf.random.set_seed(1234)
model_r = Sequential(
    [
        Dense(120, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.1)),
        Dense(40, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.1)),
        Dense(6, activation="linear")
    ], name=None
)

model_r.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=SparseCategoricalCrossentropy(from_logits=True)
)

model_r.fit(X_train, y_train, epochs=1000)

model_r.summary()

Epoch 1/1000


2024-07-15 20:37:58.743062: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 1s 18ms/step - loss: 4.4215
Epoch 2/1000
13/13 [==============================] - 0s 13ms/step - loss: 1.6719
Epoch 3/1000
13/13 [==============================] - 0s 13ms/step - loss: 1.3285
Epoch 4/1000
13/13 [==============================] - 0s 15ms/step - loss: 1.0885
Epoch 5/1000
13/13 [==============================] - 0s 15ms/step - loss: 0.9646
Epoch 6/1000
13/13 [==============================] - 0s 13ms/step - loss: 0.8939
Epoch 7/1000
13/13 [==============================] - 0s 13ms/step - loss: 0.8918
Epoch 8/1000
13/13 [==============================] - 0s 13ms/step - loss: 0.8202
Epoch 9/1000
13/13 [==============================] - 0s 13ms/step - loss: 0.7941
Epoch 10/1000
13/13 [==============================] - 0s 14ms/step - loss: 0.7747
Epoch 11/1000
13/13 [==============================] - 0s 13ms/step - loss: 0.7695
Epoch 12/1000
13/13 [==============================] - 0s 13ms/step - loss: 0.7594
Epoch 13/1000
13/13 [=====

In [29]:
model_predict_r = lambda Xl: np.argmax(tf.nn.softmax(model_r.predict(Xl)), axis = 1)
training_cerr_reg = eval_cat_err(y_train, model_predict_r(X_train))
cv_cerr_reg = eval_cat_err(y_cv, model_predict_r(X_cv))
print("Test categorization error (regularized): ", eval_cat_err(y_test, model_predict_r(X_test)))

print(f"categorization error, training, regularized: {training_cerr_reg:0.3f}, simple model, {training_cerr_simple:0.3f}, complex model: {training_cerr_complex:0.3f}" )
print(f"categorization error, cv,       regularized: {cv_cerr_reg:0.3f}, simple model, {cv_cerr_simple:0.3f}, complex model: {cv_cerr_complex:0.3f}" )

7/7 [==============================] - 0s 5ms/step


2024-07-15 20:41:32.592470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 0s 10ms/step
Test categorization error (regularized):  0.055
categorization error, training, regularized: 0.070, simple model, 0.052, complex model: 0.010
categorization error, cv,       regularized: 0.080, simple model, 0.065, complex model: 0.090


#### Iterate to find optimal regularization value

In [ ]:
tf.random.set_seed(1234)
lambda_ = [0.0, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3]
cv_err = [] # minimise this
for l in lambda_:
    model = Sequential(
        [
            Dense(120, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l)),
            Dense(40, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l)),
            Dense(6, activation="linear")
        ]
    )

    model.compile(
        loss = SparseCategoricalCrossentropy(from_logits=True),
        optimizer = Adam(learning_rate = 0.01)
    )

    model.fit(X_train, y_train, epochs=1000)

    y_pred = model.predict(X_cv)
    yhat = np.argmax(tf.nn.softmax(y_pred), axis = 1)

    cv_err.append(eval_cat_err(y_cv, yhat))

plt.plot(lambda_, cv_err)
plt.xscale("log")
plt.xlabel("lambda")
plt.ylabel("cv error")
plt.show()

print("Best lambda: ", lambda_[np.argmin(cv_err)])